In [1]:
!pip install streamlit -q
!pip install langchain-groq -q
!pip install langchain-core -q
!pip install pyngrok -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/5

In [2]:
%%writefile mood_food_prediction.py
import os
import pandas as pd
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field
from pyngrok import ngrok
import subprocess

# Set up the Streamlit page
st.set_page_config(page_title="Food Suggestion System", page_icon="🍴", layout="wide")

# Define the title and description
st.title("🍴 Food Suggestion System")
st.markdown("""
    Welcome to the Food Suggestion System! 🌟
    Enter your mood, current weather, and preference (veg or non-veg) below, and we'll recommend some delicious food for you.
    Just fill out the fields and click 'Get Suggestion' to receive personalized recommendations!
""")

# Get the API key from environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize the language model if API key is available
if not GROQ_API_KEY:
    st.error("API key is not set. Please configure the API key in the environment variables.")
else:
    try:
        # Initialize the language model with the API key
        llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY)
    except Exception as e:
        st.error(f"Failed to initialize the language model: {e}")

    # Define the Pydantic classes
    class FoodSuggestion(BaseModel):
        Food_Recommendation: str = Field(description="Suggested food based on mood, weather, and preference")

    class Response(BaseModel):
        output: FoodSuggestion

    # Function to display the response
    def display_food_suggestion(response: Response):
        if isinstance(response.output, FoodSuggestion):
            suggestion = response.output.Food_Recommendation
            st.write(f"### Food Recommendation: {suggestion}")
        else:
            st.error("Unknown response type received.")

    # Structure the model to return output based on the schema
    try:
        structured_llm = llm.with_structured_output(Response)
    except Exception as e:
        st.error(f"Error in structuring the language model output: {e}")

    # Create input form
    with st.form(key='input_form'):
        st.header("Please Enter Your Details")
        mood = st.text_input("Your Mood", placeholder="e.g., happy, sad, excited")
        weather = st.text_input("Current Weather", placeholder="e.g., sunny, cloudy, rainy")
        preference = st.selectbox("Food Preference", ["non-veg", "veg"])
        submit_button = st.form_submit_button(label="Get Suggestion")

    # Submit button action
    if submit_button:
        if not mood or not weather:
            st.error("Please enter both mood and weather before submitting.")
        else:
            try:
                # Construct the input query
                input_text = (f"Suggest {preference} food for a person who is feeling {mood} "
                              f"and the weather is {weather}.")

                # Invoke the model and display the result
                response = structured_llm.invoke(input_text)
                display_food_suggestion(response)
            except Exception as e:
                if 'tool_use_failed' in str(e):
                    st.error("Please try to submit again. Ensure your input is clear and valid.")
                else:
                    st.error(f"Unable to process the query: {e}")

Writing mood_food_prediction.py


In [3]:
from pyngrok import ngrok
import subprocess

ngrok.set_auth_token("2jhoNI1jfLmSpnrRlJ3hzjinWOz_6D3Wi4PDE4vVZrsYPRBY")
ngrok_tunnel = ngrok.connect(8501)
print("Public URL:", ngrok_tunnel.public_url)

subprocess.Popen(["streamlit", "run", "mood_food_prediction.py"])

import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Ngrok tunnel stopped.")

Public URL: https://3a61-34-87-144-106.ngrok-free.app
Ngrok tunnel stopped.
